# Importing Libraries

In [ ]:
!pip install pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 5.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 13.7 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824028 sha256=cc3b66f8bf573e300f2e08c70e199a540d87260dc9b829dc1cb1ca8fb8384412
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [ ]:
spark = SparkSession.builder.appName('AmazonReviews').getOrCreate()

In [ ]:
import pyspark
from pyspark.context import SparkContext 

from pyspark import SparkConf 
sc = SparkContext.getOrCreate(SparkConf().setMaster("local[4]"))

# Connecting The Collab With Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import sklearn

# Loading The Dataset

In [ ]:
df = spark.read.csv('/content/drive/MyDrive/Amazon_Unlocked_Mobile.csv', header=True, inferSchema=True)

# Displaying 1st 5 Data

In [ ]:
# display the dataset
df.show(5)

+--------------------+----------+------+------+--------------------+------------+
|        Product Name|Brand Name| Price|Rating|             Reviews|Review Votes|
+--------------------+----------+------+------+--------------------+------------+
|"""CLEAR CLEAN ES...|   Samsung|199.99|     5|I feel so LUCKY t...|           1|
|"""CLEAR CLEAN ES...|   Samsung|199.99|     4|nice phone, nice ...|           0|
|"""CLEAR CLEAN ES...|   Samsung|199.99|     5|        Very pleased|           0|
|"""CLEAR CLEAN ES...|   Samsung|199.99|     4|It works good but...|           0|
|"""CLEAR CLEAN ES...|   Samsung|199.99|     4|Great phone to re...|           0|
+--------------------+----------+------+------+--------------------+------------+
only showing top 5 rows



# Checking Null Values

In [ ]:
from pyspark.sql.functions import isnan, when, count, col

# Count the number of null values in each column

df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+------------+----------+-----+------+-------+------------+
|Product Name|Brand Name|Price|Rating|Reviews|Review Votes|
+------------+----------+-----+------+-------+------------+
|           0|     64376|    6|   379|     78|       11531|
+------------+----------+-----+------+-------+------------+



we have 0 null values in product name column

we have 64376 null values in brand name column

we have 6 null values in price column

we have 379 null values in rating column

we have 78 null values in reviews column

we have 11531 null values in review votes column

# Replacing Nulll Value

In [ ]:
# Fill null values with a default value

df = df.fillna({'Price': 0.0, 'Brand Name': 'Unknown', 'Reviews': 0, 'Rating': 0, 'Review Votes': 0})

# Verify that there are no more null values

df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+------------+----------+-----+------+-------+------------+
|Product Name|Brand Name|Price|Rating|Reviews|Review Votes|
+------------+----------+-----+------+-------+------------+
|           0|         0|    0|     0|      0|           0|
+------------+----------+-----+------+-------+------------+



we have replaced the null value with default value in each column

as we have 0 null value in product name column we are not going to do anything

we replaced the null values in Price column with 0.0

we replaced the null values in Brand Name column with Unknown

we replaced the null values in Reviews column with 0

we replaced the null value in Rating column with 0

we relaced the null value in Review Votes column with 0

# NO OF RECORDS

In [ ]:
# total value

df.count()

413848

we have 4,13,848 records

# NO OF COLUMNS

In [ ]:
# total column

len(df.columns)

6

we have 6 columns

# COLUMN NAMES

In [ ]:
# column name

df.columns

['Product Name', 'Brand Name', 'Price', 'Rating', 'Reviews', 'Review Votes']

In [ ]:
# no of record grouped by product name

df.groupBy("Product Name").count().show()

+--------------------+-----+
|        Product Name|count|
+--------------------+-----+
|Apple iPhone 4 A1...|  330|
|Apple iPhone 6s 1...|  163|
|"BLU Studio M HD ...|   47|
|BlueCosmo Iridium...|    1|
|"Cellphones Unloc...|   40|
|CNPGD® All-in-1 S...|  261|
|Flip Phone Unlock...|    2|
|H2O Nano SIM Card...|    1|
|LG H955 Unlocked ...|   44|
|LG LS670 OPTIMUS ...|    1|
|"LG Nexus 5 D820 ...|   62|
|4G-Unlocked Huawe...|    7|
|Apple iPhone 5C Y...|   10|
|Apple iPhone 5s U...|   10|
|Apple iPhone 6 Pl...|  176|
|Apple iPhone SE 6...|   12|
|ASUS ZenFone 2 Un...|   10|
|Blackberry 9530 S...|  484|
|BlackBerry Torch ...|   14|
|BLU Win JR Smartp...|   42|
+--------------------+-----+
only showing top 20 rows



330 people has bought apple iphone

1 person has bought LG LS670 OPTIMUS

In [ ]:
# no of record grouped by brand name

df.groupBy("Brand Name").count().show()

+--------------------+-----+
|          Brand Name|count|
+--------------------+-----+
|              DOOGEE|   97|
|                 H2O|    1|
|             Getnord|    5|
|                Kata|   40|
|                P710|   30|
| Android 4.1 - In...|   36|
|               Nokia|16086|
|                4GB"|    1|
|             LandRum|    6|
|             Ulefone|  141|
|            px phone|   84|
|               JIAKE|   32|
|                13MP|  123|
|          Jelly Bean|   42|
|8gb - Internation...|  138|
|         AeroAntenna|    1|
|      Android 4.4 KK| 1390|
|                Doro|   21|
|             Maxwest|   15|
|                 htc|  203|
+--------------------+-----+
only showing top 20 rows



16086 people has bought nokia brand

1 person has bought AeroAntenna , H2O brand

In [ ]:
# phone with the price >= 2000

price_lesser_than_1000= df.filter(df['Price'] > 2000).show()

+--------------------+----------+-----+------+--------------------+------------+
|        Product Name|Brand Name|Price|Rating|             Reviews|Review Votes|
+--------------------+----------+-----+------+--------------------+------------+
|BlueCosmo Iridium...|   Iridium| 2598|     5|These folks are g...|           0|
|"Huawei Ascend P7...|    Huawei| 2066|     1|This phonesoftwar...|           0|
|"Huawei Ascend P7...|    Huawei| 2066|     5|       great product|           0|
|"Huawei Ascend P7...|    Huawei| 2066|     5|All very good, ex...|           0|
|"Huawei Ascend P7...|    Huawei| 2066|     5|Super productTwo ...|           0|
|"Huawei Ascend P7...|    Huawei| 2066|     1|item were not as ...|           3|
|"Huawei Ascend P7...|    Huawei| 2066|     5|Great phone, grea...|           1|
|Lenovo S8 S898T 5...|    Lenovo| 2224|     3|good phone. I am ...|           0|
|Lenovo S8 S898T 5...|    Lenovo| 2224|     4|Very good product...|           0|
|Lenovo S8 S898T 5...|    Le

maximum every phone has rating 5

so it's best to buy phone at the rate greater than 2000 price

In [ ]:
# phone with highest and lowest price

from pyspark.sql.functions import min , max

df.select(max('Price'),min('Price')).show()

+----------+-----------------+
|max(Price)|       min(Price)|
+----------+-----------------+
|  verykool| 1 GHZ Dual Core |
+----------+-----------------+



verykool mobile phone has the highest price

1 GHZ Dual Core mobile phone has the lowest price

In [ ]:
# phone with 5 rating

rat_five= df.filter(df['Rating'] == 5).show()

+--------------------+----------+------+------+--------------------+------------+
|        Product Name|Brand Name| Price|Rating|             Reviews|Review Votes|
+--------------------+----------+------+------+--------------------+------------+
|"""CLEAR CLEAN ES...|   Samsung|199.99|     5|I feel so LUCKY t...|           1|
|"""CLEAR CLEAN ES...|   Samsung|199.99|     5|        Very pleased|           0|
|"""CLEAR CLEAN ES...|   Samsung|199.99|     5|I originally was ...|           0|
|"""CLEAR CLEAN ES...|   Samsung|199.99|     5|This is a great p...|           0|
|"""CLEAR CLEAN ES...|   Samsung|199.99|     5|These guys are th...|           2|
|"""CLEAR CLEAN ES...|   Samsung|199.99|     5|Ordered this phon...|           1|
|"""CLEAR CLEAN ES...|   Samsung|199.99|     5|I was able to get...|           6|
|"""CLEAR CLEAN ES...|   Samsung|199.99|     5|I brought this ph...|           0|
|"""CLEAR CLEAN ES...|   Samsung|199.99|     5|the phone was gre...|           0|
|"""CLEAR CLEAN 

samsung product has the highest rating - 5

so it's best to buy samsung product

In [ ]:
# phone with 4 rating

rat_five= df.filter(df['Rating'] == 4).show()

+--------------------+----------+------+------+--------------------+------------+
|        Product Name|Brand Name| Price|Rating|             Reviews|Review Votes|
+--------------------+----------+------+------+--------------------+------------+
|"""CLEAR CLEAN ES...|   Samsung|199.99|     4|nice phone, nice ...|           0|
|"""CLEAR CLEAN ES...|   Samsung|199.99|     4|It works good but...|           0|
|"""CLEAR CLEAN ES...|   Samsung|199.99|     4|Great phone to re...|           0|
|"""CLEAR CLEAN ES...|   Samsung|199.99|     4|I love the phone....|           1|
|"""CLEAR CLEAN ES...|   Samsung|199.99|     4|The battery was o...|           0|
|"""CLEAR CLEAN ES...|   Samsung|199.99|     4|pros-beautiful sc...|           0|
|"""CLEAR CLEAN ES...|   Samsung|199.99|     4|Phone good just a...|           0|
|"""CLEAR CLEAN ES...|   Samsung|199.99|     4|Phone's speaker l...|           0|
|((Unlocked))Black...|   Unknown| 269.1|     4|             I liked|           0|
|((Unlocked))Bla

In [ ]:
# phone with 3 rating

rat_five= df.filter(df['Rating'] == 3).show()

+--------------------+----------+------+------+--------------------+--------------------+
|        Product Name|Brand Name| Price|Rating|             Reviews|        Review Votes|
+--------------------+----------+------+------+--------------------+--------------------+
|"""CLEAR CLEAN ES...|   Samsung|199.99|     3|It's battery life...|                   0|
|"""CLEAR CLEAN ES...|   Samsung|199.99|     3|My fiance had thi...|                   0|
|"""CLEAR CLEAN ES...|   Samsung|199.99|     3|unfortunately Spr...|                   0|
|"""CLEAR CLEAN ES...|   Samsung|199.99|     3|the reasons for t...|                   0|
|((Unlocked))Black...|   Unknown| 269.1|     3|Ad advertised as ...|                   0|
|"(LANDVO) 5.0"" C...|       HTM| 69.99|     3|Valid for Movilne...|                   0|
|"(LANDVO) 5.0"" C...|       HTM| 69.99|     3|The phone works g...|                   0|
|"[XMAS DEAL] [New...|    Jethro| 79.99|     3|the charger did n...|                   0|
|"[XMAS DE

In [ ]:
# phone with 2 rating

rat_five= df.filter(df['Rating'] == 2).show()

+--------------------+----------+------+------+--------------------+------------+
|        Product Name|Brand Name| Price|Rating|             Reviews|Review Votes|
+--------------------+----------+------+------+--------------------+------------+
|"""CLEAR CLEAN ES...|   Samsung|199.99|     2|The charging port...|           0|
|"""CLEAR CLEAN ES...|   Samsung|199.99|     2|Phone looks good ...|           0|
|"""CLEAR CLEAN ES...|   Samsung|199.99|     2|Had this phone be...|           0|
|"""CLEAR CLEAN ES...|   Samsung|199.99|     2|One of the phones...|           0|
|((Unlocked))Black...|   Unknown| 269.1|     2|when i got phone ...|           0|
|"(LANDVO) 5.0"" C...|       HTM| 69.99|     2|Delivery was fast...|           0|
|"(LANDVO) 5.0"" C...|       HTM| 69.99|     2|When I first got ...|           0|
|"[XMAS DEAL] [New...|    Jethro| 79.99|     2|I like the FM rad...|           0|
|"[XMAS DEAL] [New...|    Jethro| 79.99|     2|does not work wel...|           1|
|"[XMAS DEAL] [N

In [ ]:
# phone with 1 rating

rat_five= df.filter(df['Rating'] == 1).show()

+--------------------+----------+------+------+--------------------+------------+
|        Product Name|Brand Name| Price|Rating|             Reviews|Review Votes|
+--------------------+----------+------+------+--------------------+------------+
|"""CLEAR CLEAN ES...|   Samsung|199.99|     1|I already had a p...|           1|
|"""CLEAR CLEAN ES...|   Samsung|199.99|     1|I'm really disapp...|           1|
|"""CLEAR CLEAN ES...|   Samsung|199.99|     1|I purchased this ...|          19|
|"""CLEAR CLEAN ES...|   Samsung|199.99|     1|was not in good c...|           0|
|"""CLEAR CLEAN ES...|   Samsung|199.99|     1|Just... not good....|           0|
|"(LANDVO) 5.0"" C...|       HTM| 69.99|     1|Worked OK for awh...|           0|
|"[XMAS DEAL] [New...|    Jethro| 79.99|     1|More complicated ...|           2|
|"[XMAS DEAL] [New...|    Jethro| 79.99|     1|phone reception p...|           1|
|"[XMAS DEAL] [New...|    Jethro| 79.99|     1|I was contacting ...|           1|
|"[XMAS DEAL] [N

In [ ]:
# no of record grouped by reviews

df.groupBy("Reviews").count().show()

+--------------------+-----+
|             Reviews|count|
+--------------------+-----+
|It was really bad...|    2|
|I took this phone...|    1|
|On point and fair...|    2|
|After only three ...|    1|
|Good phone..i jus...|    1|
|I bought the phon...|    3|
|Great condition a...|    4|
|the iphone 3gs is...|    3|
|Exactly as I expe...|    2|
|After having a fe...|    1|
|seller was great ...|    1|
|Do NOT get this p...|    1|
|The phone is very...|    1|
|Got to my house e...|    1|
|did not function ...|    3|
|I am happy with m...|    2|
|Having a great ti...|    2|
|My iphone 4 arriv...|    2|
|good items, the c...|    2|
|It's an iPhone, s...|    1|
+--------------------+-----+
only showing top 20 rows



In [ ]:
from pyspark.sql.functions import when
from pyspark.sql.functions import lit

In [ ]:
df.withColumn("sentiment", \
   when((df.Rating > 3), lit("positive")) \
     .when((df.Rating < 3), lit("negative")) \
     .otherwise(lit("neutral")) \
  ).show()

+--------------------+----------+------+------+--------------------+------------+---------+
|        Product Name|Brand Name| Price|Rating|             Reviews|Review Votes|sentiment|
+--------------------+----------+------+------+--------------------+------------+---------+
|"""CLEAR CLEAN ES...|   Samsung|199.99|     5|I feel so LUCKY t...|           1| positive|
|"""CLEAR CLEAN ES...|   Samsung|199.99|     4|nice phone, nice ...|           0| positive|
|"""CLEAR CLEAN ES...|   Samsung|199.99|     5|        Very pleased|           0| positive|
|"""CLEAR CLEAN ES...|   Samsung|199.99|     4|It works good but...|           0| positive|
|"""CLEAR CLEAN ES...|   Samsung|199.99|     4|Great phone to re...|           0| positive|
|"""CLEAR CLEAN ES...|   Samsung|199.99|     1|I already had a p...|           1| negative|
|"""CLEAR CLEAN ES...|   Samsung|199.99|     2|The charging port...|           0| negative|
|"""CLEAR CLEAN ES...|   Samsung|199.99|     2|Phone looks good ...|           0

In [ ]:
df.filter(df['Reviews'] == "great phone").show()

+--------------------+----------+------+------+-----------+------------+
|        Product Name|Brand Name| Price|Rating|    Reviews|Review Votes|
+--------------------+----------+------+------+-----------+------------+
|Apple Iphone 4 - ...|   Unknown|    NA|     5|great phone|           0|
|Apple iPhone 4 16...|     Apple|208.79|     5|great phone|           0|
|Apple iPhone 4 16...|     Apple|208.79|     5|great phone|           0|
|Apple iPhone 4 32...|     Apple| 99.99|     5|great phone|           0|
|Apple iPhone 4S 3...|     Apple|209.48|     5|great phone|           0|
|Apple iPhone 4S 6...|     Apple|   114|     5|great phone|           0|
|Apple iPhone 4S 6...|     Apple|   114|     5|great phone|           0|
|Apple iPhone 4s a...|   Unknown|159.99|     5|great phone|           0|
|Apple iPhone 5 Un...|     Apple|   265|     5|great phone|           0|
|Apple iPhone 5 Un...|     Apple|   309|     5|great phone|           0|
|Apple iPhone 5 Un...|     Apple|   309|     5|grea

In [ ]:
df.filter(df['Reviews'] == "not good").show()

+--------------------+----------+------+------+--------+------------+
|        Product Name|Brand Name| Price|Rating| Reviews|Review Votes|
+--------------------+----------+------+------+--------+------------+
|4 Inch Touch Scre...|   Unknown|  23.9|     1|not good|           0|
|4 Inch Touch Scre...|   Unknown|  23.9|     1|not good|           0|
|Apple iPhone 5 Un...|     Apple|   309|     1|not good|           0|
|Apple iPhone 5 Un...|     Apple|314.95|     1|not good|           0|
|Apple iPhone 5 Un...|     Apple|314.95|     1|not good|           0|
|Apple iPhone 6 Pl...|   Unknown|699.95|     2|not good|           1|
|Apple iPhone 6 Pl...|     Apple|   615|     2|not good|           1|
|Apple iPhone 6 Pl...|     Apple|   605|     2|not good|           1|
|Apple iPhone 6 Pl...|     Apple|   519|     2|not good|           1|
|Apple iPhone 6 Pl...|   Unknown|   490|     2|not good|           1|
|BLU Studio 5.0 HD...|       BLU|107.98|     1|not good|           0|
|BLU Studio 5.0 HD..

In [ ]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, StringIndexer
from pyspark.ml.classification import NaiveBayes, LogisticRegression, RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.pipeline import Pipeline
from pyspark.mllib.evaluation import MulticlassMetrics


In [ ]:
#Naive Bayes classifier
tokenizer = Tokenizer(inputCol='Review Votes', outputCol='words')
remover = StopWordsRemover(inputCol=tokenizer.getOutputCol(), outputCol='filtered')
hashingTF = HashingTF(inputCol=remover.getOutputCol(), outputCol='rawFeatures', numFeatures=1000)
idf = IDF(inputCol=hashingTF.getOutputCol(), outputCol='features')
labelIndexer = StringIndexer(inputCol='Rating', outputCol='label', handleInvalid='keep')

nb = NaiveBayes()

pipeline = Pipeline(stages=[tokenizer, remover, hashingTF, idf, labelIndexer, nb])

(trainingData, testData) = df.randomSplit([0.7, 0.3], seed=123)

model = pipeline.fit(trainingData)

predictions = model.transform(testData)

evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='accuracy')
accuracy = evaluator.evaluate(predictions)

print(f'Accuracy: {accuracy}')


Accuracy: 0.5200615047738653


In [ ]:
# Calculate evaluation metrics
metrics = MulticlassMetrics(predictions.select('prediction', 'label').rdd)
prec = metrics.weightedPrecision
rec = metrics.weightedRecall
f1 = metrics.weightedFMeasure()


# Print evaluation metrics
print(f'Precision: {prec}')
print(f'Recall: {rec}')
print(f'F1 Score: {f1}')


Precision: 0.4379434522510633
Recall: 0.5200615047738654
F1 Score: 0.39322363287054907


In [ ]:
#Logistic regression classifier
lr = LogisticRegression(maxIter=10, regParam=0.01)

pipeline = Pipeline(stages=[tokenizer, remover, hashingTF, idf, labelIndexer, lr])

(trainingData, testData) = df.randomSplit([0.7, 0.3], seed=123)

model = pipeline.fit(trainingData)

predictions = model.transform(testData)

evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='accuracy')
accuracy = evaluator.evaluate(predictions)

print(f'Accuracy: {accuracy}')

# Calculate evaluation metrics
metrics = MulticlassMetrics(predictions.select('prediction', 'label').rdd)
prec = metrics.weightedPrecision
rec = metrics.weightedRecall
f1 = metrics.weightedFMeasure()


# Print evaluation metrics
print(f'Precision: {prec}')
print(f'Recall: {rec}')
print(f'F1 Score: {f1}')

Accuracy: 0.5396560884895909
Precision: 0.48746599312436023
Recall: 0.5396560884895908
F1 Score: 0.3901890112525952


In [ ]:
#Random forest classifier
rf = RandomForestClassifier(numTrees=10)

pipeline = Pipeline(stages=[tokenizer, remover, hashingTF, idf, labelIndexer, rf])

(trainingData, testData) = df.randomSplit([0.7, 0.3], seed=123)

model = pipeline.fit(trainingData)

predictions = model.transform(testData)

evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='accuracy')
accuracy = evaluator.evaluate(predictions)

print(f'Random forest classifier Accuracy: {accuracy}')

# Calculate evaluation metrics
metrics = MulticlassMetrics(predictions.select('prediction', 'label').rdd)
prec = metrics.weightedPrecision
rec = metrics.weightedRecall
f1 = metrics.weightedFMeasure()


# Print evaluation metrics
print(f'Precision: {prec}')
print(f'Recall: {rec}')
print(f'F1 Score: {f1}')

Random forest classifier Accuracy: 0.5223961100645639
Precision: 0.4173603011272721
Recall: 0.5223961100645639
F1 Score: 0.3586813433651473


In [ ]:
#Decision Tree classifier
from pyspark.ml.classification import DecisionTreeClassifier

dt = DecisionTreeClassifier(labelCol='label', featuresCol='features')

pipeline = Pipeline(stages=[tokenizer, remover, hashingTF, idf, labelIndexer, dt])

(trainingData, testData) = df.randomSplit([0.7, 0.3], seed=123)

model = pipeline.fit(trainingData)

predictions = model.transform(testData)

evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='accuracy')
accuracy = evaluator.evaluate(predictions)

print(f'Accuracy: {accuracy}')

# Calculate evaluation metrics
metrics = MulticlassMetrics(predictions.select('prediction', 'label').rdd)
prec = metrics.weightedPrecision
rec = metrics.weightedRecall
f1 = metrics.weightedFMeasure()


# Print evaluation metrics
print(f'Precision: {prec}')
print(f'Recall: {rec}')
print(f'F1 Score: {f1}')

Accuracy: 0.5290698610507334
Precision: 0.3314578038398621
Recall: 0.5290698610507334
F1 Score: 0.3674930330395204


In [ ]:
# K-means clustering
from pyspark.ml.clustering import KMeans

kmeans = KMeans().setK(2).setSeed(1)
pipeline = Pipeline(stages=[tokenizer, remover, hashingTF, idf, labelIndexer, kmeans])

(trainingData, testData) = df.randomSplit([0.7, 0.3], seed=123)

model = pipeline.fit(trainingData)

predictions = model.transform(testData).withColumn("prediction", col("prediction").cast("double"))

evaluator = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction', metricName='accuracy')
accuracy = evaluator.evaluate(predictions)

print(f'Accuracy: {accuracy}')

# Calculate evaluation metrics
metrics = MulticlassMetrics(predictions.select('prediction', 'label').rdd)
prec = metrics.weightedPrecision
rec = metrics.weightedRecall
f1 = metrics.weightedFMeasure()


# Print evaluation metrics
print(f'Precision: {prec}')
print(f'Recall: {rec}')
print(f'F1 Score: {f1}')

Accuracy: 0.5218889371910673
Precision: 0.27237244815661843
Recall: 0.5218889371910673
F1 Score: 0.35793800406490034
